# CT2FE example 02
## CT dataset to tethraedra mesh and FE model solution in Calculix
11-01-2021

The preprocessing of this example is performed in 3D Slicer using the extension [SlicerSegmenterMesher](https://github.com/lassoan/SlicerSegmentMesher#tutorial). <br />
This extension of 3D Slicer allows the creation of volumetric meshes from segmented 3D data using Cleaver2 or TetGen. <br />
The full SlicerSegmentMesher tutorial can be found here:
https://github.com/lassoan/SlicerSegmentMesher#tutorial  <br />
For info on the solver see the Calculix homepage:
http://www.calculix.de/

#### Activate CT2FE kernel in Jupyter:
`conda env list`

`conda activate CT2FE`

`python -m ipykernel install --user --name CT2FE --display-name "Python (CT2FE)"`

## Basic configurations

In [21]:
import os
import numpy as np
import tifffile
import dxchange
import matplotlib
import matplotlib.pyplot as plt

astropy module not found


In [2]:
matplotlib.rcParams['figure.dpi'] = 100

In [3]:
import logging
logging.basicConfig(level=logging.INFO)

#### Inputs and parameters definition

In [4]:
# input_file = 'test_data/example_02/masked_8bit/scan-sample_15-3-crop_c0001.tif'
# filename_in, ext = os.path.split(input_file)
# vs = [0.0065, 0.0065, 0.0065] # voxel size

In [5]:
Fiij_exe = '/home/gianthk/Applications/Fiji.app/ImageJ-linux64'
Fiji_exe_stack = Fiij_exe + ' -macro /home/gianthk/Applications/Fiji.app/macros/FolderOpener_virtual.ijm '

## Pre-processing
The following steps are performed in [3D Slicer](https://www.slicer.org/)
- [x] **3D rotate** (**Transforms** module). Apply a rigid rotation (manual) along P-A and R-L planes to align the strut vertically. 
![](test_data/example_02/3DSlicer_transform.png)
- [x] **Orient Scalar Volume** (**Converters** module): Orientation: axial (NO)
- [x] **Crop Volume** (**Converters** module): limit the model size and flattens the strut top and bottom
- [x] **3D global threshold** (Kittler-Illingworth)
    - [x] Smooth with closing method (fill holes)
    - [x] Invert (twice) and remove small islands
    - [x] Add the large pores segmented separately
    - [x] Median smoothing
- [x] **Segmenter Mesher**
    - Feature scaling: 0.8
    - Sampling rate: 0.5
    - Rate of change el size: 0.4



## Modify VTK unstructured grid mesh; generate CalculiX input

We will need to modify the mesh in the following way: <br />

- [ ] Add points sets for boundary nodes: NODES_S, NODES_N, NODES_E, NODES_W, NODES_T, NODES_B
- [ ] Add elemets sets for boundary elements: ELEMS_S, ELEMS_N, ELEMS_E, ELEMS_W, ELEMS_T, ELEMS_B
- [ ] ...

We use the module *meshio* to import the mesh. 

In [1]:
import meshio

In [5]:
filename = '/home/gianthk/Data/TOMCAT/Kaya/D_single_h1h2_3DSlicer/D_single.vtk'
mesh = meshio.read(filename)
vars(mesh)

{'points': array([[303.579   , 306.325   , 104.5993  ],
        [297.65    , 312.2541  ,  98.670296],
        [303.7763  , 324.3659  , 104.6251  ],
        ...,
        [ 83.271996, 193.76599 , 362.8216  ],
        [ 83.311   , 193.76599 , 365.02863 ],
        [ 83.590996, 193.76599 , 367.2356  ]], dtype=float32),
 'cells': [<meshio CellBlock, type: tetra, num cells: 742468>],
 'point_data': {},
 'cell_data': {'labels': [array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]},
 'field_data': {},
 'point_sets': {},
 'cell_sets': {},
 'gmsh_periodic': None,
 'info': None}

In [33]:
# find model boundaries
model_coors_max = np.amax(mesh.points, 0) # [microns?]
model_coors_min = np.amin(mesh.points, 0)

In [70]:
# find boundary points
NODES_E = np.where(mesh.points[:,0] == model_coors_min[0])[0]
NODES_W = np.where(mesh.points[:,0] == model_coors_max[0])[0]
NODES_S = np.where(mesh.points[:,1] == model_coors_min[1])[0]
NODES_N = np.where(mesh.points[:,1] == model_coors_max[1])[0]
NODES_B = np.where(mesh.points[:,2] == model_coors_min[2])[0]
NODES_T = np.where(mesh.points[:,2] == model_coors_max[2])[0]

In [86]:
# add dictionary of point sets
mesh.point_sets = {
    'NODES_N': NODES_N,
    'NODES_S': NODES_S,
    'NODES_W': NODES_W,
    'NODES_E': NODES_E,
    'NODES_T': NODES_T,
    'NODES_B': NODES_B
    }

In [248]:
# find boundary cells
ELEMS_E = np.array([]).astype('int')
ELEMS_W = np.array([]).astype('int')
ELEMS_S = np.array([]).astype('int')
ELEMS_N = np.array([]).astype('int')
ELEMS_T = np.array([]).astype('int')
ELEMS_B = np.array([]).astype('int')

for node_e in NODES_E:
    ELEMS_E = np.append(ELEMS_E, np.where(np.any(mesh.cells[0][1] == node_e, axis=1)))

for node_w in NODES_W:
    ELEMS_W = np.append(ELEMS_W, np.where(np.any(mesh.cells[0][1] == node_w, axis=1)))

for node_s in NODES_S:
    ELEMS_S = np.append(ELEMS_S, np.where(np.any(mesh.cells[0][1] == node_s, axis=1)))

for node_n in NODES_N:
    ELEMS_N = np.append(ELEMS_N, np.where(np.any(mesh.cells[0][1] == node_n, axis=1)))

for node_t in NODES_T:
    ELEMS_T = np.append(ELEMS_T, np.where(np.any(mesh.cells[0][1] == node_t, axis=1)))

for node_b in NODES_B:
    ELEMS_B = np.append(ELEMS_B, np.where(np.any(mesh.cells[0][1] == node_b, axis=1)))

In [250]:
np.unique(ELEMS_B)+1

array([     1,      2,      3, ..., 697625, 697665, 697666])

In [251]:
mesh.cell_sets = {
    'ELEMS_N': np.unique(ELEMS_N)+1,
    'ELEMS_S': np.unique(ELEMS_S)+1,
    'ELEMS_W': np.unique(ELEMS_W)+1,
    'ELEMS_E': np.unique(ELEMS_E)+1,
    'ELEMS_T': np.unique(ELEMS_T)+1,
    'ELEMS_B': np.unique(ELEMS_B)+1,
    'SET1': np.arange(1, len(mesh.cells[0][1]))
    }

In [238]:
len(mesh.cells[0][1])

742468

In [242]:
np.arange(1,10)
NODES_T[0]

47558

In [208]:
ELEMS_B = np.array([]).astype('int')
for node_b in NODES_B:
    ELEMS_B = np.append(ELEMS_B, np.where(np.any(mesh.cells[0][1] == node_b, axis=1).tolist()))

In [210]:
mesh.cell_sets = {
    'ELEMS_B': np.unique(ELEMS_B)
    }

In [221]:
# empty cell_sets
mesh.cell_sets = {}

## Add material mapping

In [252]:
vars(mesh)

{'points': array([[303.579   , 306.325   , 104.5993  ],
        [297.65    , 312.2541  ,  98.670296],
        [303.7763  , 324.3659  , 104.6251  ],
        ...,
        [ 83.271996, 193.76599 , 362.8216  ],
        [ 83.311   , 193.76599 , 365.02863 ],
        [ 83.590996, 193.76599 , 367.2356  ]], dtype=float32),
 'cells': [<meshio CellBlock, type: tetra, num cells: 742468>],
 'point_data': {},
 'cell_data': {'labels': [array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]},
 'field_data': {},
 'point_sets': {'NODES_N': array([1363]),
  'NODES_S': array([150191]),
  'NODES_W': array([1515]),
  'NODES_E': array([30737]),
  'NODES_T': array([ 47558,  47564,  47565, ..., 162186, 162187, 162189]),
  'NODES_B': array([     1,      3,      4, ..., 154049, 154050, 154051])},
 'cell_sets': {'ELEMS_N': array([  1831,   1832,   1833,   1834,   1835,   1836,   1837,   1838,
         600382, 600383, 600384]),
  'ELEMS_S': array([590950, 590951, 590955, 590956, 590958, 590959, 590964, 590971,
         5909

## Write Abaqus output

In [253]:
# write Abaqus mesh using meshio
filename_out = '/home/gianthk/Data/TOMCAT/Kaya/D_single_h1h2_3DSlicer/CalculiX/D_single.inp'
meshio.abaqus.write(filename_out, mesh, float_fmt=".6e")

TypeError: object of type 'numpy.int64' has no len()

array([3, 4, 0, 1], dtype=int32)

In [57]:
mesh.points

array([[303.579   , 306.325   , 104.5993  ],
       [297.65    , 312.2541  ,  98.670296],
       [303.7763  , 324.3659  , 104.6251  ],
       ...,
       [ 83.271996, 193.76599 , 362.8216  ],
       [ 83.311   , 193.76599 , 365.02863 ],
       [ 83.590996, 193.76599 , 367.2356  ]], dtype=float32)

In [254]:
# test Abaqus mesh read
test_filename = '/home/gianthk/PycharmProjects/CT2FE/test_data/example_01/pippo.inp'
tmp = meshio.abaqus.read(test_filename)

In [269]:
# vars(tmp)

In [256]:
len(tmp.cells)

101

In [260]:
type(tmp.cell_sets['SET3'])

list

#### Visualize template CalculiX analisys definition

In [54]:
!more {template_inp}

** ---------------------------------------------------
** The Step module defines the analysis steps and associated output requests. Mo
re info at:
** https://abaqus-docs.mit.edu/2017/English/SIMACAECAERefMap/simacae-m-Sim-sb.ht
m#simacae-m-Sim-sb
** ---------------------------------------------------
*STEP
*STATIC
*BOUNDARY
NODES_B, 1, 3, 0.0
*BOUNDARY
NODES_T, 3, 3, -0.01
*NODE FILE
U
*EL FILE
S
*END STEP


In [55]:
input_file = 'test_data/example_01/masked_8bit_cap/scan-sample_15-3-crop_c_0000.tiff'
filename_out, ext_out = os.path.splitext(output_file)
template_inp = './temp.inp'
%run stack2Abaqus.py {input_file} {output_file} -vs {vs[0]} {vs[1]} {vs[2]} -k NODE ELEMENT NSET PROPERTY -p 'prop_bone.inp' 'prop_steel.inp' -pr 1:250 254:255 -t {template_inp} -v True

INFO:root:Data loaded with size 34 x 80 x 80
INFO:root:User material properties defined.
INFO:root:Reading Abaqus template file ./temp.inp
INFO:root:Data written to file test_data/example_01/pippo.inp


## Solve FE model in calculix
The following section assumes that you have the Calculix solver installed and accessible with the command `ccx_2.17` or for multithread option `ccx_2.17_MT` <br />
The multithread option in CalculiX is activated by defining the number of threads (default=1) used for the calculation.<br />
This is done by setting the env variable *OMP_NUM_THREADS* with:
`export OMP_NUM_THREADS=8`



In [263]:
# !export OMP_NUM_THREADS=8; ccx_2.17_MT {filename_out}

### Convert Calculix output to Paraview
The following sections assume that you have **ccx2paraview** and **Paraview** installed and working.<br /> For more info visit: <br />
https://www.paraview.org/ <br />
https://github.com/calculix/ccx2paraview

In [264]:
import ccx2paraview

In [270]:
filename_out = '/home/gianthk/Data/TOMCAT/Kaya/D_single_h1h2_3DSlicer/CalculiX/D_single.inp'
filename_out_base, ext_out = os.path.splitext(filename_out)
filename_out_base

'/home/gianthk/Data/TOMCAT/Kaya/D_single_h1h2_3DSlicer/CalculiX/D_single'

In [271]:
ccx2paraview.Converter(filename_out_base + '.frd', ['vtk']).run()

Visualize results in Paraview

In [65]:
!paraview filename_out_base + '.vtk'

![](test_data/example_01/masked_8bit_cap1.png)